# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
data.info()
data.describe()
#нет пустых значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.5+ MB


,user
count,289955.000000
mean,9852.460447
std,5692.355041
min,1.000000
25%,4935.000000
50%,9838.000000
75%,14769.000000
max,19718.000000


In [4]:
countries = ['Spain', 'Portugal', 'France', 'Italy', 'Belgium']

In [5]:
#оставили в датасете нужные страны
data = data.loc[data['country'].isin(countries)]
data.head()

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal


In [6]:
data_by_invoice = data.groupby(['user'])['artist'].apply(lambda x: ','.join(x)).reset_index()
data_by_invoice

,user,artist
0,6,"lily allen,kanye west,sigur rós,pink floyd,ste..."
1,12,"queen,the beatles,lynyrd skynyrd,neil young,th..."
2,36,"m.i.a.,fugazi,johnny cash,clint mansell,elton ..."
3,47,"pj harvey,radiohead,zero 7,einstürzende neubau..."
4,74,"in extremo,nine inch nails,billy talent,panic!..."
...,...,...
1414,19690,"pink floyd,the beatles,the white stripes,depec..."
1415,19693,"air,simon & garfunkel,pixies,serge gainsbourg,..."
1416,19697,"underworld,pendulum,the prodigy,sufjan stevens..."
1417,19704,"keane,pinback,coldplay,travis,calexico,doves,d..."


In [7]:
dummy_data = data_by_invoice['artist'].str.get_dummies(',')
data = pd.concat([data_by_invoice, dummy_data], axis=1)
data.head()

,user,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,6,"lily allen,kanye west,sigur rós,pink floyd,ste...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12,"queen,the beatles,lynyrd skynyrd,neil young,th...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,36,"m.i.a.,fugazi,johnny cash,clint mansell,elton ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,47,"pj harvey,radiohead,zero 7,einstürzende neubau...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,74,"in extremo,nine inch nails,billy talent,panic!...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
